git clone https://github.com/MCvin/python4gate.git

# GATE output formats and Python

## GATE can output information in different formats:

### From actors, examples:
* SimulationStatisticActor : store number of events, tracks, steps in **ASCII**
* DoseActor : store absorbed dose in **ASCII, ROOT for 1D and 2D**, and **.hdr, .mhd for 3D**
* EnergySpectrumActor: store energy distribution in **ROOT**
* PhaseSpaceActor: store the particle's type, position, direction, energy, ... in **ROOT, IAEA, NumPy**

See https://opengate.readthedocs.io/en/latest/tools_to_interact_with_the_simulation_actors.html

### From imaging "systems" (scanner, CT, SPECT, PET, ...):
* to record interactions: hits, singles, coincidences in **ASCII, binary, ROOT, NumPy**
* to save images: projections and sinograms in **.hdr, .mhd**

See https://opengate.readthedocs.io/en/latest/data_output_management.html

## In summary, when using GATE you will produce one of these output formats:
* ASCII output
* Binary output
* Images (.mhd, .hdr) output
* ROOT tree output
* NumPy tree output (new!)

# Demonstration with the GATE application Ex_beam

#### We will first have a look at a GATE example to understand these outputs:
* open a terminal and go to the folder gate_outputs/Ex_beam
* look at the different files and open mac/main.mac
* look at all the sections and in particular to the ouputs section
* run the example with  
`Gate --qt mac/main.mac`
* check the output folder and the different files produced

#### We will now analyse each output with this Python notebook !

# GATE ASCII output

## 1D DoseActor (along Z)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Load data from txt files
d_edep = np.loadtxt('Ex_Beam/output/1D-depth-Edep.txt')
d_uncert = np.loadtxt('Ex_Beam/output/1D-depth-Edep-Uncertainty.txt')

## Plot depth dose and uncertainty

In [ ]:
# Declare a single figure (one row, one column)
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(12, 6), facecolor='w')

# generate x for the plots
x = np.linspace(0, len(d_edep), len(d_edep))

# First curve, gamma depth in green
c1 = ax.plot(x, d_edep, 'g-', label='edep', linewidth=2)

# Second curve, gamma uncertainty in blue, share the same x axis, but use a different y axis
ax2 = ax.twinx()
c2 = ax2.plot(x, d_uncert, 'b-', label='$\sigma$ (uncertainty)')

# Add the legend and the title
fig.legend(bbox_to_anchor=(1,1), bbox_transform=ax.transAxes)
ax.set_xlabel('Distance')
ax.set_ylabel('Deposited energy in MeV')
f = ax2.set_ylabel('Uncertainty')

# GATE image output

## 3D DoseActor - format mhd/raw

In [ ]:
import SimpleITK as sitk

img_dose = sitk.ReadImage('Ex_Beam/output/3D-Edep.mhd')
arr_dose = sitk.GetArrayFromImage(img_dose)
print('Image size = ', arr_dose.shape)
print('Image min and max: ',  np.amin(arr_dose), np.amax(arr_dose))

In [ ]:
arr_dose

## Exercise
* make the same plot (depth dose) as previous section with this 3D data

In [ ]:
# complete here ...

# GATE list of interactions

## Binary format
See https://opengate.readthedocs.io/en/latest/data_output_management.html#hits-file-gatehits-dat-bin

In [ ]:
import numpy as np

# dtype line format for "scanner" GATE system binary output (26 columns)
dt=np.dtype([('run_ID', np.int32), ('event_ID', np.int32), ('primary_ID', np.int32), ('source_ID', np.int32), 
             ('level1_ID', np.int32), ('level2_ID', np.int32), ('level3_ID', np.int32), ('level4_ID', np.int32), 
             ('level5_ID', np.int32), ('layer0_ID', np.int32), ('layer1_ID', np.int32), 
             ('time', np.float64), ('energy', np.float64), ('range', np.float64), 
             ('x', np.float64), ('y', np.float64), ('z', np.float64), 
             ('G4code', np.int32), ('particle_ID', np.int32), ('mother_particle_ID', np.int32), ('photon_mother_ID', np.int32), 
             ('n_Compt', np.int32), ('n_Rayl', np.int32), 
             ('process', np.dtype('S8')), ('vol_Compt', np.dtype('S8')), ('vol_Rayl', np.dtype('S8'))])

data=np.fromfile('Ex_Beam/output/list-Hits.bin', dtype=dt)
print('Number of particles:', len(data))
data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot the energy histogram
x = data['energy']

n, bins, patches = plt.hist(x, bins=100, range=[0,0.5], alpha=0.75)

## Exercise  
* select and print only data corresponding to the process 'phot'
* plot the corresponding energy histogram
* explain the energy spectrum (increase the number of primaries if necessary)

In [ ]:
# complete here ...

## GATE ROOT Tree output

In [ ]:
import uproot

f = uproot.open('Ex_Beam/output/tree.hits.root')

# all trees, branches and leaves names are accessible through the method keys()
print('Trees in the file:')
print(f.keys())
print()
print('Leaves (variables) in the Tree:')
print(f['tree'].keys())

# all variables are accessible through the method arrays()
data = f['tree'].arrays()

print()
print('Number of particles:', len(data[b'trackID']))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Plot the energy histogram
x = data[b'edep']

n, bins, patches = plt.hist(x, bins=100, range=[0,0.5], alpha=0.75)

In [ ]:
from matplotlib.colors import LogNorm

# Plot the particle positions
x = data[b'posX']
y = data[b'posY']

fig = plt.scatter(x, y, alpha=0.75)
#fig = plt.hist2d(x, y, bins=100, norm=LogNorm())

## GATE Numpy Tree output - New !
GATE simulations can now output information on interactions occuring in a detector in a new format: the NumPy .npy Python format. This as several advantages:
* data is easily open in Python.
* data is saved as a [Structured Arrays](https://docs.scipy.org/doc/numpy/user/basics.rec.html) which makes the access to variables very easy.

In [ ]:
import numpy as np

data = np.load('Ex_Beam/output/tree.hits.npy')

print('Number of particles:', len(data))
data

## Exercise
Make the same data analysis as with the ROOT output:
* select and plot the histogram of the particle energies
* select and plot the particle positions

In [ ]:
# complete here ...